### Funkcja do pobierania danych tabelarycznych.

In [1]:
def adverts(pages=1):
    
    import bs4
    from urllib.request import urlopen 
    from bs4 import BeautifulSoup as soup 
    import pandas as pd
    
    my_url = 'https://www.olx.pl/nieruchomosci/mieszkania/wynajem/warszawa/'
    links = []
    base = []

    for i in range(pages+1):
        
        if pages == 0:
            break
            
        elif i >= 1:
            url = my_url + '?page=' + str(i)
            uClient = urlopen(url)
            page_html = uClient.read()
            page_soup = soup(page_html, 'html.parser')
            containers = page_soup.find_all('tr', {'class' : 'wrap'})
            for cont in containers:
                link = cont.tr.td.a['href']
                if 'olx' in link:
                    links.append(link)
                    
    rows = []
    for link in list(set(links)):
        try:
            ad = urlopen(link).read()
            ad_soup = soup(ad)
            
            added_class = ad_soup.find_all('li', {'class':'offer-bottombar__item'})
            added = added_class[0].text.strip().split(', ')[1]
            
            localization_class = ad_soup.find_all('div', {'class':'offer-user__address'})
            city = localization_class[0].text.split()[0]
            district = localization_class[0].text.split()[2]

            price_class = ad_soup.find_all('div', {'class':'pricelabel'})
            price = price_class[0].text.strip().split('\n')[0]
            price = int(price.replace(' ', '').replace('zł',''))

            by_class = ad_soup.find_all('ul', {'class':'offer-details'})
            by = by_class[0].a.strong.text

            level_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            level = level_class[1].a.text.strip().split('\n')[1]

            furniture_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            furniture = furniture_class[2].a.text.strip().replace('\n', ' ').split()[1]
            furniture
            if furniture == 'Tak':
                furniture = 1
            else:
                furniture = 0

            building_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            building = building_class[3].a.text.split()[2]

            surface_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            surface = int(surface_class[4].text.strip().replace('\n', ' ').split()[1])

            rooms_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            rooms = rooms_class[5].text.strip().replace('\n', ' ').split()[2]

            rent_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            rent = int(rent_class[6].text.strip().replace('\n', ' ').split()[2])
            
            rows.append([link, city, district, price, by, level, furniture, 
                         building, surface, rooms, rent])
        except Exception:
            pass
    print('pobrano dane z', len(rows), 'ogłoszeń.')
    return rows

In [2]:
from datetime import datetime
now = datetime.now()
print(now.strftime('%d/%m/%Y %H:%M:%S'))

rows = adverts(25)

11/11/2020 10:24:11
pobrano dane z 650 ogłoszeń.


### Funkcja automatycznie dodająca nowe wiersze do istiejącego arkusza Excela (skopiowane ze StackOverflow).

In [3]:
def append_df_to_excel(filename, df, sheet_name='Sheet1', startrow=None,
                       truncate_sheet=False, 
                       **to_excel_kwargs):
    """
    Append a DataFrame [df] to existing Excel file [filename]
    into [sheet_name] Sheet.
    If [filename] doesn't exist, then this function will create it.

    Parameters:
      filename : File path or existing ExcelWriter
                 (Example: '/path/to/file.xlsx')
      df : dataframe to save to workbook
      sheet_name : Name of sheet which will contain DataFrame.
                   (default: 'Sheet1')
      startrow : upper left cell row to dump data frame.
                 Per default (startrow=None) calculate the last row
                 in the existing DF and write to the next row...
      truncate_sheet : truncate (remove and recreate) [sheet_name]
                       before writing DataFrame to Excel file
      to_excel_kwargs : arguments which will be passed to `DataFrame.to_excel()`
                        [can be dictionary]

    Returns: None

    (c) [MaxU](https://stackoverflow.com/users/5741205/maxu?tab=profile)
    """
    from openpyxl import load_workbook

    # ignore [engine] parameter if it was passed
    if 'engine' in to_excel_kwargs:
        to_excel_kwargs.pop('engine')

    writer = pd.ExcelWriter(filename, engine='openpyxl')

    # Python 2.x: define [FileNotFoundError] exception if it doesn't exist 
    try:
        FileNotFoundError
    except NameError:
        FileNotFoundError = IOError


    try:
        # try to open an existing workbook
        writer.book = load_workbook(filename)
        
        # get the last row in the existing Excel sheet
        # if it was not specified explicitly
        if startrow is None and sheet_name in writer.book.sheetnames:
            startrow = writer.book[sheet_name].max_row

        # truncate sheet
        if truncate_sheet and sheet_name in writer.book.sheetnames:
            # index of [sheet_name] sheet
            idx = writer.book.sheetnames.index(sheet_name)
            # remove [sheet_name]
            writer.book.remove(writer.book.worksheets[idx])
            # create an empty sheet [sheet_name] using old index
            writer.book.create_sheet(sheet_name, idx)
        
        # copy existing sheets
        writer.sheets = {ws.title:ws for ws in writer.book.worksheets}
    except FileNotFoundError:
        # file does not exist yet, we will create it
        pass

    if startrow is None:
        startrow = 0

    # write out the new sheet
    df.to_excel(writer, sheet_name, startrow=startrow, **to_excel_kwargs)

    # save the workbook
    writer.save()

In [4]:
import pandas as pd

base = pd.DataFrame(rows, 
                    columns = ['link', 'miasto', 'dzielnica', 'cena', 'od', 'poziom', 
                               'umeblowanie','zabudowa','powierzchnia', 'pokoje', 'czynsz dodatkowo'])
    
append_df_to_excel('excel2.xlsx', base, index=False, header=False)

### Funkcja do pobierania danych ze wskazanych dzielnic. 
Dzielnice, w których jest najmniej ogłoszeń (wniosek wyciągnięty po analizie EDA).

In [1]:
def adverts_districsts(districts, pages=1):
    '''
    districts: list, at least one with the following:
        'Wesoła', 'Rembertów', 'Wawer', 'Wilanów', 'Ursus', 'Praga-Północ', 'Żoliborz'
    '''
    import bs4
    from urllib.request import urlopen 
    from bs4 import BeautifulSoup as soup 
    import pandas as pd

    my_url = 'https://www.olx.pl/nieruchomosci/mieszkania/wynajem/warszawa/'
    links = []
    rows = []
    base = []
    district_dict = dict([('533', 'Wesoła'), ('361', 'Rembertów'), ('383', 'Wawer'), ('357', 'Wilanów'), 
                          ('371', 'Ursus'), ('379', 'Praga-Północ'), ('363', 'Żoliborz')]) # słownik dzielnic i odopiwadających im kluczy id na stronie
    keys = [key for key, value in district_dict.items() if value in districts] # lista kluczy wybranych w funkcji dzielnic
    
    for k in keys:
        for i in range(pages+1):
            if pages == 0:
                break

            elif i >= 1:
                url = my_url + '?search%5Bdistrict_id%5D=' + str(k) + '&page=' + str(i)
                if str(k) in url:
                    uClient = urlopen(url)
                    page_html = uClient.read()
                    page_soup = soup(page_html, 'html.parser')
                    containers = page_soup.find_all('tr', {'class' : 'wrap'})
                    for cont in containers:
                        link = cont.tr.td.a['href']
                        if 'olx' in link:
                            links.append(link)

    for link in list(set(links)):
        try:
            ad = urlopen(link).read()
            ad_soup = soup(ad)

            added_class = ad_soup.find_all('li', {'class':'offer-bottombar__item'})
            added = added_class[0].text.strip().split(', ')[1]

            localization_class = ad_soup.find_all('div', {'class':'offer-user__address'})
            city = localization_class[0].text.split()[0]
            district = localization_class[0].text.split()[2]

            price_class = ad_soup.find_all('div', {'class':'pricelabel'})
            price = price_class[0].text.strip().split('\n')[0]
            price = int(price.replace(' ', '').replace('zł',''))

            by_class = ad_soup.find_all('ul', {'class':'offer-details'})
            by = by_class[0].a.strong.text

            level_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            level = level_class[1].a.text.strip().split('\n')[1]

            furniture_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            furniture = furniture_class[2].a.text.strip().replace('\n', ' ').split()[1]
            furniture
            if furniture == 'Tak':
                furniture = 1
            else:
                furniture = 0

            building_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            building = building_class[3].a.text.split()[2]

            surface_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            surface = int(surface_class[4].text.strip().replace('\n', ' ').split()[1])

            rooms_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            rooms = rooms_class[5].text.strip().replace('\n', ' ').split()[2]

            rent_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            rent = int(rent_class[6].text.strip().replace('\n', ' ').split()[2])

            rows.append([link, city, district, price, by, level, furniture, 
                         building, surface, rooms, rent])
        except Exception:
            pass

    print('pobrano dane z', len(rows), 'ogłoszeń.')
    return rows

In [28]:
from datetime import datetime
now = datetime.now()
print(now.strftime('%d/%m/%Y %H:%M:%S'))

rows = adverts_districsts(districts=['Wesoła', 'Rembertów', 'Wawer', 'Wilanów', 'Ursus',], pages=1)

03/01/2021 11:57:43
pobrano dane z 145 ogłoszeń.


In [29]:
import pandas as pd

base = pd.DataFrame(rows, 
                    columns = ['link', 'miasto', 'dzielnica', 'cena', 'od', 'poziom', 
                               'umeblowanie','zabudowa','powierzchnia', 'pokoje', 'czynsz dodatkowo'])
#append_df_to_excel(filename='ogloszenia.xlsx', df=base, sheet_name='dzielnice', index=False, header=True)

In [30]:
base['dzielnica'].value_counts()

Wawer        31
Ursus        31
Rembertów    29
Wesoła       27
Włochy       27
Name: dzielnica, dtype: int64

### Funkcja pobierające opisy
Pobieram dane tabelaryczne, które są potrzebne do podstawienia do modelu regresji  oraz opis i tytuł. Następnie obliczę składniki resztowe które będą punktem odniesienia dla modelu analizującego tekst.

In [80]:
def adverts(pages=1):
    
    import bs4
    from urllib.request import urlopen 
    from bs4 import BeautifulSoup as soup 
    import pandas as pd
    
    my_url = 'https://www.olx.pl/nieruchomosci/mieszkania/wynajem/warszawa/'
    links = []
    base = []

    for i in range(pages+1):
        
        if pages == 0:
            break
            
        elif i >= 1:
            url = my_url + '?page=' + str(i)
            uClient = urlopen(url)
            page_html = uClient.read()
            page_soup = soup(page_html, 'html.parser')
            containers = page_soup.find_all('tr', {'class' : 'wrap'})
            for cont in containers:
                link = cont.tr.td.a['href']
                if 'olx' in link:
                    links.append(link)
                    
    rows = []
    for link in list(set(links)):
        try:
            ad = urlopen(link).read()
            ad_soup = soup(ad)
            
            localization_class = ad_soup.find_all('div', {'class':'offer-user__address'})
            district = localization_class[0].text.split()[2]

            price_class = ad_soup.find_all('div', {'class':'pricelabel'})
            price = price_class[0].text.strip().split('\n')[0]
            price = int(price.replace(' ', '').replace('zł',''))

            by_class = ad_soup.find_all('ul', {'class':'offer-details'})
            by = by_class[0].a.strong.text

            level_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            level = level_class[1].a.text.strip().split('\n')[1]

            building_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            building = building_class[3].a.text.split()[2]

            surface_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            surface = int(surface_class[4].text.strip().replace('\n', ' ').split()[1])

            rent_class = ad_soup.find_all('li', {'class':'offer-details__item'})
            rent = int(rent_class[6].text.strip().replace('\n', ' ').split()[2])
            
            # dane tekstowe
            title_class = ad_soup.find_all('div', {'class':'offer-titlebox'})
            title = title_class[0].h1.text.strip() 
            
            description_class = ad_soup.find_all('div', {'class':'clr lheight20 large'})
            description = description_class[0].text.strip().replace('\r\n', ' ')
            
            rows.append([district, price, by, level, building,
                         surface, rent, title, description])
        except Exception:
            pass
    print('pobrano dane z', len(rows), 'ogłoszeń.')
    return rows

In [79]:
import bs4
from urllib.request import urlopen 
from bs4 import BeautifulSoup as soup 
import pandas as pd

link = r'https://www.olx.pl/oferta/dizajnerska-kawalerka-na-chmielnej-CID3-IDHXtY7.html#bea34531db;promoted'

ad = urlopen(link).read()
ad_soup = soup(ad)

description_class = ad_soup.find_all('div', {'class':'clr lheight20 large'})
description = description_class[0].text.strip().replace('\r\n', ' ')

title_class = ad_soup.find_all('div', {'class':'offer-titlebox'})
title = title_class[0].h1.text.strip()

In [81]:
rows = adverts()

pobrano dane z 30 ogłoszeń.


In [82]:
import pandas as pd
base = pd.DataFrame(rows, 
                    columns = ['dzielnica', 'cena', 'od', 'poziom', 'zabudowa',
                               'powierzchnia', 'czynsz dodatkowo', 'tytuł', 'opis'])

In [83]:
base.head()

,dzielnica,cena,od,poziom,zabudowa,powierzchnia,czynsz dodatkowo,tytuł,opis
0,Wola,2000,Osoby prywatnej,Parter,Blok,36,550,"Mieszkanie do wynajęcia, Twarda centrum, W-wa,...",Do wynajęcia mieszkanie (36m2) po generalnym r...
1,Mokotów,2600,Biuro / Deweloper,Parter,Blok,64,1,Wynajmę mieszkanie - Stefana Bryły - MOKOTÓW -...,Wynajmę 3 pokojowe mieszknie na ul. Stanisława...
2,Mokotów,2750,Biuro / Deweloper,5,Apartamentowiec,45,1,LUX apartament 2 pok ok 45m2 przy ul. Cybernet...,LUX apartament 2 pok ok 45m2 przy ul. Cybernet...
3,Mokotów,2300,Osoby prywatnej,Powyżej 10,Blok,50,1,"Mieszkanie 2 pokoje+wydzielona sypialnia, przy...","Dzień dobry, zapraszam do wynajmu dwupokojoweg..."
4,Wola,1500,Osoby prywatnej,3,Blok,28,400,Wynajmę mieszkanie ul.Ogrodowa,Wynajmę mieszkanie/kawalerkę - 28 m². Lokal d...
